In [34]:
abstract class Record
case class EmptyRecord()
case class PostRecord (val id: String, val favs: Int) {
  override def toString = "PostRecord: " + id + " " + favs
}
case class VoteRecord (val postid: String, val ups: Int, val downs: Int) {
  def + (that: VoteRecord) = 
  new VoteRecord(this.postid, 
     this.ups + that.ups,
     this.downs + that.downs)
  
  override def toString = "VoteRecord: " + postid + " " + ups + "/" + downs
}
case class UserRecord (userid: String, rep: Int)
case class OutRecord(val favs: Int, val ups: Int, val downs: Int) {
  def this(pRec: PostRecord, vRec: VoteRecord) = 
  this(pRec.favs, vRec.ups, vRec.downs)
  def + (that: OutRecord) =
    new OutRecord(this.favs + that.favs,
      this.ups + that.ups,
      this.downs + that.downs)
}

Name: Compile Error
Message: <console>:29: error: reference to PostRecord is ambiguous;
it is imported twice in the same scope by
import INSTANCE.PostRecord
and import INSTANCE.PostRecord
         def this(pRec: PostRecord, vRec: VoteRecord) = 
                        ^
<console>:29: error: reference to VoteRecord is ambiguous;
it is imported twice in the same scope by
import INSTANCE.VoteRecord
and import INSTANCE.VoteRecord
         def this(pRec: PostRecord, vRec: VoteRecord) = 
                                          ^
StackTrace: 

In [2]:
def postParser(line: String): PostRecord = {
  val id_patt = "Id=\"(\\d+)\"".r
  val favs_patt = "FavoriteCount=\"(\\d+)\"".r
  def parser(xs: List[String], id: String, favs: Int):
  PostRecord = xs match {
    case List() => new PostRecord(id, favs)
    case (x :: xs1) => x match {
      case id_patt(new_id) => parser(xs1, new_id, favs)
      case favs_patt(new_favs) => parser(xs1, id, new_favs.toInt)
      case _ => parser(xs1, id, favs)
    }
  }
  parser(line.split("\\s+").toList, "", 0)
}

In [3]:
def voteParser(line: String): VoteRecord = {
  val id_patt = "PostId=\"(\\d+)\"".r
  val vid_patt = "VoteTypeId=\"(\\d+)\"".r
  def parser(xs: List[String], id: String, up: Int, down: Int):
  VoteRecord = xs match {
    case List() => new VoteRecord(id,up,down)
    case (x :: xs1) => x match {
      case id_patt(new_id) => parser(xs1,new_id,up,down)
      case vid_patt(vid) =>
        if (vid == "2") {
          parser(xs1,id,1,0)
        } else if (vid == "3") {
          parser(xs1,id,0,1)
        }   else {
          parser(xs1,id,0,0)
        }
      case _ => parser(xs1,id,up,down)
    }
  }
  parser(line.split("\\s+").toList,"",0,0)
}

In [4]:
val postLine = """<row Id="5" PostTypeId="1" CreationDate="2014-05-13T23:58:30.457" Score="7" ViewCount="296" Body="&lt;p&gt;I've always been interested in machine      learning, but I can't figure out one thing about starting out with a simple &quot;Hello World&quot; example - how can I avoid hard-coding behavior     ?&lt;/p&gt;&#xA;&#xA;&lt;p&gt;For example, if I wanted to &quot;teach&quot; a bot how to avoid randomly placed obstacles, I couldn't just use relat     ive motion, because the obstacles move around, but I don't want to hard code, say, distance, because that ruins the whole point of machine learning     .&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Obviously, randomly generating code would be impractical, so how could I do this?&lt;/p&gt;&#xA;" OwnerUserId="5" Las     tActivityDate="2014-05-14T00:36:31.077" Title="How can I do simple machine learning without hard-coding behavior?" Tags="&lt;machine-learning&gt;"      AnswerCount="1" CommentCount="1" FavoriteCount="1" ClosedDate="2014-05-14T14:40:25.950" />"""
postParser(postLine)

PostRecord(5,1)

In [5]:
val voteLine = """<row Id="1" PostId="1" VoteTypeId="2" CreationDate="2014-05-13T00:00:00.000" />"""
voteParser(voteLine)

1: 1/0

In [6]:
val voteFile = sc.textFile("resources/Votes.xml")
voteFile take(5)

Array(﻿<?xml version="1.0" encoding="utf-8"?>, <votes>, "  <row Id="1" PostId="1" VoteTypeId="2" CreationDate="2014-05-13T00:00:00.000" />", "  <row Id="2" PostId="1" VoteTypeId="2" CreationDate="2014-05-13T00:00:00.000" />", "  <row Id="3" PostId="3" VoteTypeId="2" CreationDate="2014-05-13T00:00:00.000" />")

In [18]:
val votes = voteFile filter(_.contains("<row ")) map(voteParser)

In [10]:
val voteSummary0 = votes.reduce(_ + _)

In [13]:
println(("#" * 80 + "\nTotal UpVotes: %s, Total DownVotes: %s")
  .format(voteSummary0.ups, voteSummary0.downs))

################################################################################
Total UpVotes: 18117, Total DownVotes: 1126


In [19]:
val posts = sc.textFile("resources/Posts.xml") filter (_ contains("<row ")) map(postParser)

In [20]:
println(("#" * 80 + "\nTotal Posts: %s, Total Votes: %s")
  .format(posts.count(), votes.count()))

################################################################################
Total Posts: 6980, Total Votes: 24317


In [27]:
val votesByPost =
  posts map(p => (p.id, p)) join(
    votes map(v => (v.postid, v))
  )

In [44]:
val deb = votesByPost map({
        case (id: String, (post, vote)) =>
          (id, new OutRecord(post.favs, vote.ups, vote.downs))
      }) map(_._1)
println(deb takeOrdered(1))

[Ljava.lang.String;@67132c73


In [48]:
val joe = List(("a",1))
joe.unzip

(List(a),List(1))

In [51]:
import scala.xml._

In [54]:
val page =     
  <html>
      <head>
        <title>Hello XHTML world</title>
      </head>
      <body>
        <h1>Hello world</h1>
        <p><a href="http://scala-lang.org/">Scala</a> talks XHTML</p>
      </body>
    </html>



In [66]:
println(page \ "head" \ "title" toString)

<title>Hello XHTML world</title>


In [76]:
val elem = XML.loadString(postLine)


Name: org.xml.sax.SAXParseException
Message: Attribute name "Las" associated with an element type "row" must be followed by the ' = ' character.
StackTrace: com.sun.org.apache.xerces.internal.util.ErrorHandlerWrapper.createSAXParseException(ErrorHandlerWrapper.java:203)
com.sun.org.apache.xerces.internal.util.ErrorHandlerWrapper.fatalError(ErrorHandlerWrapper.java:177)
com.sun.org.apache.xerces.internal.impl.XMLErrorReporter.reportError(XMLErrorReporter.java:400)
com.sun.org.apache.xerces.internal.impl.XMLErrorReporter.reportError(XMLErrorReporter.java:327)
com.sun.org.apache.xerces.internal.impl.XMLScanner.reportFatalError(XMLScanner.java:1438)
com.sun.org.apache.xerces.internal.impl.XMLDocumentFragmentScannerImpl.scanAttribute(XMLDocumentFragmentScannerImpl.java:1531)
com.sun.org.apache.xerces.internal.impl.XMLDocumentFragmentScannerImpl.scanStartElement(XMLDocumentFragmentScannerImpl.java:1320)
com.sun.org.apache.xerces.internal.impl.XMLDocumentScannerImpl$ContentDriver.scanRootElem